In [6]:
import urllib.request
from bs4 import BeautifulSoup
import os
import re
import json
import codecs

json_dic = {"item": "a","page": "a","turn": "0","unit":"a","position": "a","interest_num": "a","price_m2": "a","url_unit": "a",
            "broker": "a","job": "a","tel": "a","group": "a","href_broker": "a"}
json_arr=[]

j_file=open('json_data.txt','w',encoding='utf-8')

item=1
page=1
while page < 12:
    
    url_page = "https://nj.lianjia.com/xiaoqu/pukou/pg"+str(page)+"cro21/"  
    page_open=urllib.request.urlopen(url_page)
    page_read=page_open.read()
    page_bs=BeautifulSoup(page_read)
    page_bsdata = page_bs.decode("utf-8")
    
    '''
    page_file = open('web'+str(page)+'.txt','w+',encoding='utf-8')
    page_file.write(page_bsdata)
    page_file.close()
    '''

    infos = page_bs.select(".info")
    


    turn = 1    
    for info in infos:
        titles = info.select(".title")     
        for title in titles:                                    
            value_as = title.select("a")  
            for value_a in value_as:
                url = value_a.get("href")
                
                
                resp=urllib.request.urlopen(url)
                html=resp.read()
                bs=BeautifulSoup(html) 
                bsdata = bs.decode("utf-8")                                                 #不要删除从此开始
                '''
                file_house_code = open('item_code'+str(item)+'.txt','w+',encoding='utf-8')
                file_house_code.write(bsdata)
                file_house_code.close()                                                     #不要删除到此结束
                '''

                try:
                
                    item_exclude = [44,343,13,34,54,66,71,124,189,190,271,286,288,298,312]
                    if item in item_exclude:          # 44\343   \13\34\54\66\71\124\154\219\220\301\316\318\328\342                       
                        pass
                    else:
                        #print(str(item))
                        #print(url)
                        #print(str(page))
                        #print(str(turn))
                        name = bs.select("h1")
                        #print(name[0].string)
                    
                        json_dic["item"] = str(item)                                     #json格式
                        json_dic["url_unit"] = str(url)
                        json_dic["page"] = str(page)
                        json_dic["turn"] = str(turn)
                        json_dic["unit"] = name[0].string
                    
                        descs = bs.select(".detailDesc")
                        for desc in descs:
                            #print(desc.string)
                            json_dic["position"] = desc.string
                        
                        follownumber = bs.select('span[data-role="followNumber"]')
                        #print(follownumber[0].string+'位用户关注')

                        json_dic["interest_num"] = follownumber[0].string
                    
                        #此处应该有爬取的图片

                        unitprice = bs.select(".xiaoquUnitPrice")
                        unitpricedesc = bs.select(".xiaoquUnitPriceDesc")
                        #print(unitprice[0].string+'元/m2'+' '+unitpricedesc[0].string)
                    
                        json_dic["price_unit"] =unitprice[0].string
                    
                        #此处应为附近门店信息开始
 
                    
                    
                        agent = bs.select('a[class="agentName LOGCLICK LOGCLICKDATA"]')
                        agent_href= agent[0].get("href")
                        #print(agent_href)
                        agent = urllib.request.urlopen(agent_href,timeout=10)
                        ag_html=agent.read()
                        ag=BeautifulSoup(ag_html) 
                    
            
                        ag_name = ag.select(".agent-name")
                        ag_pub = ag.select(".pub-tag")
                        ag_tel = ag.select(".agent-tel")
                        ag_map = ag.select(".map-text")

                        #print(ag_name[0].string)
                        #print(ag_pub[0].string)
                        #print(ag_tel[0].string)
                        #print(ag_map[0].string)
                    
                        json_dic["href_broker"] = agent_href
                        json_dic["broker"] = ag_name[0].string
                        json_dic["job"] = ag_pub[0].string
                        json_dic["tel"] = ag_tel[0].string
                        json_dic["group"] = ag_map[0].string
                    
                    
                        j_string = json.dumps(json_dic)
                        #json_arr.append(j_string)
                        #print(j_string)
                    
                        print(item)
                        j_file.write(j_string)
                        j_file.write('\n')
                except IOError:
                    print("Error: 没有找到网页或写入文件失败")
                else:    
                    pass
                    
                turn += 1                                                        #不要删除
                item += 1
    page += 1

print('end')
j_file.close()


1
2
3
4
5
6
7
8
9
10
11
12
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
35
36
37
38
39
40
41
42
43
45
46
47
48
49
50
51
52
53
55
56
57
58
59
60
61
62
63
64
65
67
68
69
70
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
272
273
274
275
276
277
278
279
280
281
282
283
284
285
28

In [5]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-

try:
    fh = open("testfile.txt", "w")
    fh.write("这是一个测试文件，用于测试异常!!")
except IOError:
    print("Error: 没有找到文件或读取文件失败")
else:
    print("内容写入文件成功")
fh.close()

内容写入文件成功
